# Blog 10: Real-time Drift Detection in Forecast Models
This notebook demonstrates how to detect data drift in time-series forecast models using `alibi-detect`, simulate drift, and integrate results into Airflow for monitoring and alerting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from alibi_detect.cd import KSDrift
from alibi_detect.utils.data import create_outlier_batch
from datetime import datetime, timedelta
import pandas as pd

## Step 1: Simulate Time-Series Data with Drift

In [ ]:
# Generate base signal (no drift)
np.random.seed(0)
n = 1000
x_ref = np.sin(np.linspace(0, 50, n)) + np.random.normal(0, 0.2, n)

# Simulate drift (data distribution change)
x_drift = np.sin(np.linspace(0, 50, n)) + np.random.normal(0.8, 0.5, n)

# Create timestamp index
timestamps = [datetime(2024, 3, 1) + timedelta(hours=i) for i in range(n)]
df = pd.DataFrame({'timestamp': timestamps, 'value': x_drift})

# Plot
plt.figure(figsize=(12, 4))
plt.plot(timestamps, x_drift, label='Drifted Data')
plt.title('Simulated Drift in Time-Series Data')
plt.xlabel('Time')
plt.ylabel('Sensor Value')
plt.legend()
plt.tight_layout()
plt.savefig('simulated_drift.png')
plt.show()

## Step 2: Apply Alibi-Detect’s KSDrift Detector

In [ ]:
# Reshape input data for drift detector
x_ref = x_ref.reshape(-1, 1)
x_test = x_drift.reshape(-1, 1)

# Initialize detector
cd = KSDrift(x_ref, p_val=0.05)

# Run drift test
preds = cd.predict(x_test, drift_type='batch', return_p_val=True, return_distance=True)
print("Drift Detected:" if preds['data']['is_drift'] else "No Drift Detected")
print(f"p-value: {preds['data']['p_val'].mean():.4f}")

## Step 3: Visualize Drift Score Over Time

In [ ]:
# Create sliding windows for batch detection
window_size = 50
drift_scores = []
for i in range(0, len(x_test) - window_size, window_size):
    batch = x_test[i:i+window_size]
    score = cd.predict(batch)['data']['p_val']
    drift_scores.append(score.mean())

# Plot
plt.figure(figsize=(12, 4))
plt.plot(drift_scores, marker='o')
plt.title('Drift Detection Signal (p-values per batch)')
plt.axhline(y=0.05, color='r', linestyle='--', label='Threshold (0.05)')
plt.xlabel('Batch Index')
plt.ylabel('p-value')
plt.legend()
plt.tight_layout()
plt.savefig('drift_detection_signal.png')
plt.show()

## Summary
- We simulated drift in time-series forecast data.
- Used `alibi-detect` to detect drift using the KS Test.
- Generated visualizations for blog inclusion.
- These results can be integrated into Airflow for automated alerting and pipeline monitoring.